# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/06-loading-structured-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File '00-start-eva-server.ipynb' already there; not retrieving.



Note: you may need to restart the kernel to use updated packages.


Starting EVA Server ...
nohup eva_server --port 8804 > eva.log 2>&1 &


### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [2]:
# sample dataset of 2 videos
!wget -nc "https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip"

# unzip
!unzip -n bddtest.zip

File 'bddtest.zip' already there; not retrieving.



Archive:  bddtest.zip


## Load 2 BDD videos

In [3]:
cursor.execute('DROP TABLE IF EXISTS bddtest_1;').fetch_all().as_df()

cursor.execute('DROP TABLE IF EXISTS bddtest_2;').fetch_all().as_df()

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;').fetch_all().as_df()

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;').fetch_all().as_df()

0
0  Number of loaded VIDEO: 1

## Create a BDD Meta-data Table

In [4]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query).fetch_all().as_df()

Empty DataFrame
Columns: []
Index: []

## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [5]:
cursor.execute("LOAD CSV './bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;").fetch_all().as_df()
cursor.execute("LOAD CSV './bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;").fetch_all().as_df()

CSV  Number of loaded frames
0  bddtest/info/bddtest_00a395fed60c0b47.csv                     4355

## Run a Query over the Video dataset


In [6]:
cursor.execute("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
      """).fetch_all().as_df()

cursor.execute("""SELECT id, Yolo(data) 
                  FROM bddtest_1 
                  WHERE id < 3;""").fetch_all().as_df()

bddtest_1.id                                        yolo.labels   
0             0  [car, stop sign, car, car, car, car, car, car,...  \
1             1  [car, car, stop sign, car, car, car, car, car,...   
2             2  [car, stop sign, car, car, car, car, car, car,...   

                                         yolo.bboxes   
0  [[490.404541015625, 332.927001953125, 716.9006...  \
1  [[486.3106994628906, 331.76861572265625, 716.0...   
2  [[481.7199401855469, 331.24237060546875, 715.6...   

                                         yolo.scores  
0  [0.93, 0.91, 0.87, 0.82, 0.77, 0.73, 0.73, 0.5...  
1  [0.92, 0.87, 0.85, 0.76, 0.74, 0.73, 0.62, 0.5...  
2  [0.93, 0.9, 0.87, 0.79, 0.77, 0.74, 0.55, 0.54...

## Run a query over the Meta-data Table

In [7]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""").fetch_all().as_df()

bddtestmeta._row_id  bddtestmeta.id  bddtestmeta.frame_id   
0                       1            8185                     0  \
1                       2            8186                     0   
2                       3            8187                     0   
3                       4            8188                     0   
4                       5            8189                     0   
...                   ...             ...                   ...   
3396                 3397           11581                  1197   
3397                 3398           11582                  1197   
3398                 3399           11583                  1197   
3399                 3400           11584                  1197   
3400                 3401           11585                  1197   

      bddtestmeta.video_id bddtestmeta.dataset_name bddtestmeta.label   
0                        4                  bddtest               car  \
1                        4                  bddtest               car   
2                        4                  bddtest               car   
3                        4                  bddtest               car   
4                        4                  bddtest               car   
...                    ...                      ...               ...   
3396                     4                  bddtest               car   
3397                     4                  bddtest               car   
3398                     4                  bddtest               car   
3399                     4                  bddtest               car   
3400                     4                  bddtest               car   

                                  bddtestmeta.bbox  bddtestmeta.object_id  
0         [491.7783, 320.9238, 717.0901, 512.1478]                    512  
1      [928.2679, 298.47574, 1279.5381, 571.17786]                    513  
2       [816.0277, 347.52887, 1029.6998, 474.7344]                    514  
3       [811.87067, 325.91223, 948.2217, 408.2217]                    515  
4      [673.0254, 325.08084, 751.17786, 367.48267]                    516  
...                                            ...                    ...  
3396     [554.96533, 335.88916, 588.2217, 378.291]                    643  
3397     [693.8106, 350.0231, 734.5497, 395.75058]                    644  
3398  [683.83374, 345.03464, 733.71826, 389.93073]                    642  
3399     [676.351, 346.69745, 699.6305, 384.11084]                    641  
3400      [662.2171, 351.6859, 681.3395, 377.4596]                    638  

[3401 rows x 8 columns]